Imports

In [1]:
from configParser import ConfigParser, getModelName, getDatasetName

Parameters

In [2]:
experimentName = "50_hier_baseline_phase8"
numOfExperiments=20

params = {
    # Do not change for multi/hyperp experiments
    "image_path": "/data/BGNN_data/INHS_cropped", # "/data/BGNN_data/CIFAR", # "/data/BGNN_data/INHS_cropped",
    "suffix":'50', #  #'50_11', '50', None # used to get a subset cleaned_metadata file. set to None otherwise to use full metadata
    
    # dataset
    "training_count":0.64,
    "validation_count":0.16,

    # training
    "batchSize": 32, # Bigger is more stable
    "n_epochs":5000, # allow it to run for long time 4000
    "learning_rate":0.01, # Seems 0.0001 to be best value with Adam. 0.1 best for SGD
    "numOfTrials":3,
    "patience": 100,
    # temp. Should be optimized and removed
    "fc_width":200,
    "fc_layers": 1,
    "modelType":"",
    "lambda": 0.6,
    "unsupervisedOnTest": False,
    ###
    "tl_model", # Keep '18', '50', 'CIFAR', 'NIN'
}

Create file

In [3]:
cp = ConfigParser(experimentName)

cp.write(params)

'50_hier_baseline_phase8-2fe7e3d0a74b11eab9d5ac1f6b9e3b4a/params.json'

Read file

In [4]:
j = cp.read()
print(j)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 32, 'n_epochs': 5000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': True, 'downsample': True, 'downsampleOutput': 200, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'lambda': 0.6, 'unsupervisedOnTest': False, 'fc_layers': 1, 'resnet': '50', 'normalizeFromResnet': True}


Create hyperp

In [5]:
hyperp_params = {
    "dummy":[0]
#     "fc_layers":[1,2,3],
    
#     "imageDimension": [28, 56, 224],
#     "n_channels": [1, 3],
#     "kernelSize": [5, 9],
#     "useZCAWhitening": [False, True],
#     "patience": [100, 200],
#     "batchSize": [500, 1300, 2600],
#     "useHeirarchy": [True, False],
#     "resnet": ['18', '50']
#     "batchNormalize": [True]
#     "lambda": [0, 0.25, 0.75, 1, 4, 50]
}

cp.writeHyperp(hyperp_params)

'50_hier_baseline_phase8-2fe7e3d0a74b11eab9d5ac1f6b9e3b4a/hyperp_params.json'

Get experiments

In [6]:
for i in cp.getHyperpIter():
    print(i)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 32, 'n_epochs': 5000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': True, 'downsample': True, 'downsampleOutput': 200, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'lambda': 0.6, 'unsupervisedOnTest': False, 'fc_layers': 1, 'resnet': '50', 'normalizeFromResnet': True, 'dummy': 0}


Model name

In [7]:
print(getModelName(j))
print(getDatasetName(j))

tc0.640000_vc0.160000_bs32_e5000_p100_lr0.010000_hTrue_dsTrue_dso200_tfiTrue_tfio200_res50_fc1_nResTrue_unsupFalse_lmda0.6
tc0.640000_vc0.160000


Hyperp search write

In [8]:
from hyperopt import fmin, hp, rand, tpe
from hyperopt.pyll.base import scope
import math

hyperp_search_params = {
#     "kernels": [ scope.int(hp.quniform('kernels1', 8, 16, 1)),
#                 scope.int(hp.quniform('kernels2', 16, 32, 1)),
#                 scope.int(hp.qloguniform('kernels3', math.log(1), math.log(16), 1)),
#                 scope.int(hp.qloguniform('kernels4', math.log(1), math.log(16), 1)),
#                 scope.int(hp.qloguniform('kernels5', math.log(1), math.log(16), 1))],
#     "kernelSize": scope.int(hp.quniform('kernelSize', 3, 10, 1)), 
    # "useZCAWhitening": hp.choice('useZCAWhitening', [False, True]), 
#     "n_channels": scope.int(hp.choice('n_channels', [1, 3])), 
#     "imageDimension": scope.int(hp.quniform('imageDimension', 28, 420, 28)),
#     "batchSize": scope.int(hp.quniform('batchSize', 50, 550, 50)),
#     "learning_rate": hp.loguniform('learning_rate', math.log(0.0001), math.log(0.01)),
#     "patience": scope.int(hp.quniform('patience', 50, 500, 25)),
    "lambda": hp.uniform('lambda', 0.05, 1),
#     "learning_rate": hp.uniform('learning_rate', 0.0001, 0.01),
#     "downsample": hp.pchoice('downsample', [(0.7, True), (0.3, False)]),
#     "takeFromIntermediate": hp.pchoice('takeFromIntermediate', [(0.7, True), (0.3, False)]),
    "downsampleOutput": scope.int(hp.quniform('downsampleOutput', 50, 500, 50)),
    "takeFromIntermediateOutput": scope.int(hp.quniform('takeFromIntermediateOutput', 50, 500, 50)),
    "fc_layers": scope.int(hp.quniform('fc_layers', 1, 5, 1)),
    "resnet": hp.choice('resnet', ["50", "18"]),
    "numOfTrials": hp.choice('numOfTrials', [2])
}

cp.writeHyperpSearch(hyperp_search_params)

'50_hier_baseline_phase8-2fe7e3d0a74b11eab9d5ac1f6b9e3b4a/hyperp_search_params.pkl'

Hyperp search read

In [9]:
hyperp_obj = cp.getHyperpSearchObject()
print(hyperp_obj)

def objective(experiment_params):
    print(experiment_params)
    dummyObjective = 1
    return dummyObjective

fmin(objective,
     space=hyperp_obj,
     algo=rand.suggest, 
     max_evals=numOfExperiments)

{'lambda': <hyperopt.pyll.base.Apply object at 0x7fcff3835e48>, 'downsampleOutput': <hyperopt.pyll.base.Apply object at 0x7fcff38355c0>, 'takeFromIntermediateOutput': <hyperopt.pyll.base.Apply object at 0x7fcff383f048>, 'fc_layers': <hyperopt.pyll.base.Apply object at 0x7fcff383f278>, 'resnet': <hyperopt.pyll.base.Apply object at 0x7fcff383fda0>, 'numOfTrials': <hyperopt.pyll.base.Apply object at 0x7fcff384c048>, 'image_path': <hyperopt.pyll.base.Apply object at 0x7fcff3835c50>, 'suffix': <hyperopt.pyll.base.Apply object at 0x7fcff384c2b0>, 'training_count': <hyperopt.pyll.base.Apply object at 0x7fcff384c400>, 'validation_count': <hyperopt.pyll.base.Apply object at 0x7fcff384c550>, 'batchSize': <hyperopt.pyll.base.Apply object at 0x7fcff384c6a0>, 'n_epochs': <hyperopt.pyll.base.Apply object at 0x7fcff384c7f0>, 'learning_rate': <hyperopt.pyll.base.Apply object at 0x7fcff384c940>, 'patience': <hyperopt.pyll.base.Apply object at 0x7fcff384ca90>, 'useHeirarchy': <hyperopt.pyll.base.Apply o

{'batchSize': 32, 'downsample': True, 'downsampleOutput': 150, 'fc_layers': 3, 'image_path': '/data/BGNN_data/INHS_cropped', 'lambda': 0.8408290086350604, 'learning_rate': 0.01, 'n_epochs': 5000, 'normalizeFromResnet': True, 'numOfTrials': 2, 'patience': 100, 'resnet': '18', 'suffix': '50', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 350, 'training_count': 0.64, 'unsupervisedOnTest': False, 'useHeirarchy': True, 'validation_count': 0.16}
{'batchSize': 32, 'downsample': True, 'downsampleOutput': 100, 'fc_layers': 3, 'image_path': '/data/BGNN_data/INHS_cropped', 'lambda': 0.39209479510963413, 'learning_rate': 0.01, 'n_epochs': 5000, 'normalizeFromResnet': True, 'numOfTrials': 2, 'patience': 100, 'resnet': '18', 'suffix': '50', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 400, 'training_count': 0.64, 'unsupervisedOnTest': False, 'useHeirarchy': True, 'validation_count': 0.16}
{'batchSize': 32, 'downsample': True, 'downsampleOutput': 450, 'fc_layers': 3, 'ima

{'batchSize': 0,
 'downsample': 0,
 'downsampleOutput': 450.0,
 'fc_layers': 2.0,
 'image_path': 0,
 'lambda': 0.32533532049689423,
 'learning_rate': 0,
 'n_epochs': 0,
 'normalizeFromResnet': 0,
 'numOfTrials': 0,
 'patience': 0,
 'resnet': 0,
 'suffix': 0,
 'takeFromIntermediate': 0,
 'takeFromIntermediateOutput': 300.0,
 'training_count': 0,
 'unsupervisedOnTest': 0,
 'useHeirarchy': 0,
 'validation_count': 0}

Selected hyperp write

In [10]:
selected_hyperp_params = [
#     {"training_count":0.32,"validation_count":0.08,},
#     {"training_count":0.32,"validation_count":0.08,'downsample': True, 'takeFromIntermediate': False, 'takeFromIntermediateOutput':-1 },
#     {"training_count":0.16,"validation_count":0.04,'downsample': True, 'takeFromIntermediate': False, 'takeFromIntermediateOutput':-1 },
#     {"training_count":0.16,"validation_count":0.04}
    
#     {"training_count":0.64,"validation_count":0.16,'downsample': True, 'takeFromIntermediate': False, 'takeFromIntermediateOutput':-1 },
#     {"training_count":0.64,"validation_count":0.16}
    
    {'downsample': True, 'takeFromIntermediate': True, 'downsampleOutput':200, 'takeFromIntermediateOutput':200, "fc_layers": 1 }, #size
#     {'downsample': True, 'takeFromIntermediate': True, 'downsampleOutput':400, 'takeFromIntermediateOutput':400, "fc_layers": 2 }, #size
#     {'downsample': True, 'takeFromIntermediate': True, 'downsampleOutput':200, 'takeFromIntermediateOutput':200, "fc_layers": 3 }, #size
#     {'downsample': True, 'takeFromIntermediate': True, 'downsampleOutput':400, 'takeFromIntermediateOutput':400, "fc_layers": 3 }, #size

#     {'downsample': True, 'takeFromIntermediate': True }, #HGNN
#     {'downsample': True, 'takeFromIntermediate': False, 'takeFromIntermediateOutput':-1 }, #blackbox
#     {'downsample': False, 'takeFromIntermediate': True }, #baseline1
#     {'downsample': True, 'takeFromIntermediate': False }, #baseline2
#     {'downsample': False, 'takeFromIntermediate': False }, #baseline3
#     {'downsample': False, 'takeFromIntermediate': True, 'downsampleOutput':0 }, #DISCO
#     {'downsample': False, 'takeFromIntermediate': True, 'downsampleOutput':-1 }, #DSN
#     {'downsample': True, 'takeFromIntermediate': True, 'lambda':0 }, #Lambda0


]

cp.writeHyperpSelected(selected_hyperp_params)

'50_hier_baseline_phase8-2fe7e3d0a74b11eab9d5ac1f6b9e3b4a/hyperp_selected_params.pkl'

Selected hyperp read

In [11]:
for i in cp.getHyperpSelectedIter():
    print(i)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 32, 'n_epochs': 5000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': True, 'downsample': True, 'downsampleOutput': 200, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'lambda': 0.6, 'unsupervisedOnTest': False, 'fc_layers': 1, 'resnet': '50', 'normalizeFromResnet': True}
